In [1]:
import pandas as pd

In [51]:
datos=pd.read_csv('data/optimizar.csv').head()
datos.head()

,Unnamed: 0,Player,PosFantasy,Precio,VI,USG%,ORtg,DRtg,eFG%,AGE,Puntos Fantasy
0,0,joel embiid,Front,18.0,15.3,39.6,124.0,104.8,0.561,30.1,58.283487
1,1,luka doncic,Back,19.0,15.5,36.0,121.9,109.9,0.573,25.1,59.412571
2,2,giannis antetokounmpo,Front,17.5,14.9,33.0,125.6,106.9,0.623,29.4,55.168493
3,3,shai gilgeous-alexander,Back,17.0,11.9,32.9,128.6,107.7,0.567,25.8,49.569773
4,4,jalen brunson,Back,15.5,10.0,32.5,124.2,112.6,0.543,27.6,42.892610


In [52]:
datos.dtypes

Unnamed: 0          int64
Player             object
PosFantasy         object
Precio            float64
VI                float64
USG%              float64
ORtg              float64
DRtg              float64
eFG%              float64
AGE               float64
Puntos Fantasy    float64
dtype: object

In [53]:
import pandas as pd
import pulp

# Cargar los datos
datos = pd.read_csv('data/optimizar.csv').dropna()

# Crear el problema de optimización
prob = pulp.LpProblem("Optimización_Fantasy", pulp.LpMaximize)

# Crear variables de decisión
jugadores = datos['Player'].tolist()
vars_decision = pulp.LpVariable.dicts("Jugador", jugadores, lowBound=0, upBound=1, cat='Binary')

# Función objetivo
prob += pulp.lpSum([vars_decision[j] * datos.loc[datos['Player'] == j, 'Puntos Fantasy'].values[0] for j in jugadores])

# Restricciones
# Presupuesto total menor o igual a 100
prob += pulp.lpSum([vars_decision[j] * datos.loc[datos['Player'] == j, 'Precio'].values[0] for j in jugadores]) <= 100

# Exactamente 5 jugadores front y 5 back (guard)
prob += pulp.lpSum([vars_decision[j] for j in jugadores if datos.loc[datos['Player'] == j, 'PosFantasy'].values[0] == 'Front']) == 5
prob += pulp.lpSum([vars_decision[j] for j in jugadores if datos.loc[datos['Player'] == j, 'PosFantasy'].values[0] == 'Back']) == 5

# Exactamente 10 jugadores en total
prob += pulp.lpSum([vars_decision[j] for j in jugadores]) == 10

# Resolver el problema
status = prob.solve()

# Verificar si se encontró una solución
if pulp.LpStatus[status] != "Optimal":
    print("No se pudo encontrar una solución óptima.")
else:
    # Obtener resultados
    equipo_seleccionado = [j for j in jugadores if vars_decision[j].value() > 0.5]
    puntos_totales = sum(datos.loc[datos['Player'].isin(equipo_seleccionado), 'Puntos Fantasy'])
    costo_total = sum(datos.loc[datos['Player'].isin(equipo_seleccionado), 'Precio'])

    print("Equipo seleccionado:")
    for jugador in equipo_seleccionado:
        PosFantasy = datos.loc[datos['Player'] == jugador, 'PosFantasy'].values[0]
        precio = datos.loc[datos['Player'] == jugador, 'Precio'].values[0]
        puntos = datos.loc[datos['Player'] == jugador, 'Puntos Fantasy'].values[0]
        print(f"{jugador} ({PosFantasy}) - Precio: {precio}, Puntos Fantasy: {puntos}")

    print(f"\nPuntos Fantasy totales: {puntos_totales}")
    print(f"Costo total: {costo_total}")

    # Verificar las restricciones
    front_count = sum(1 for j in equipo_seleccionado if datos.loc[datos['Player'] == j, 'PosFantasy'].values[0] == 'Front')
    back_count = sum(1 for j in equipo_seleccionado if datos.loc[datos['Player'] == j, 'PosFantasy'].values[0] == 'Back')

    print(f"\nNúmero de jugadores Front: {front_count}")
    print(f"Número de jugadores Back: {back_count}")
    print(f"Número total de jugadores: {len(equipo_seleccionado)}")
    print(f"¿Se cumple la restricción de presupuesto?: {'Sí' if costo_total <= 100 else 'No'}")

# Verificar si hay inconsistencias en los datos
inconsistencias = datos[datos['PosFantasy'].isin(['Front', 'Back']) == False]
if not inconsistencias.empty:
    print("\nAdvertencia: Se encontraron jugadores con Posiciones no válidas:")
    print(inconsistencias[['Player', 'PosFantasy']])

Equipo seleccionado:
joel embiid (Front) - Precio: 18.0, Puntos Fantasy: 58.283487179487175
giannis antetokounmpo (Front) - Precio: 17.5, Puntos Fantasy: 55.16849315068493
shai gilgeous-alexander (Back) - Precio: 17.0, Puntos Fantasy: 49.56977333333333
dalano banton (Back) - Precio: 8.5, Puntos Fantasy: 28.6569
kenneth lofton jr. (Front) - Precio: 5.0, Puntos Fantasy: 28.045
jordan goodwin (Back) - Precio: 7.5, Puntos Fantasy: 28.74147058823529
jordan goodwin (Back) - Precio: 7.5, Puntos Fantasy: 28.74147058823529
kenneth lofton jr. (Front) - Precio: 5.0, Puntos Fantasy: 28.045
dalano banton (Back) - Precio: 8.5, Puntos Fantasy: 28.6569
kenneth lofton jr. (Front) - Precio: 5.0, Puntos Fantasy: 28.045

Puntos Fantasy totales: 273.67944091840747
Costo total: 99.5

Número de jugadores Front: 5
Número de jugadores Back: 5
Número total de jugadores: 10
¿Se cumple la restricción de presupuesto?: Sí


# v2

In [54]:
import pandas as pd
import pulp

# Cargar los datos
datos = pd.read_csv('data/optimizar.csv').dropna().drop_duplicates(subset=['Player'])

# Lista de jugadores lesionados
jugadores_lesionados = [
    'joel embiid',
    'luka doncic',
    'giannis antetokounmpo',
    'chimezie metu',
    'jordan goodwin',
    'shai gilgeous-alexander',
    'shaedon sharpe',
    'gordon hayward',
    'ja morant'
]

# Eliminar los jugadores lesionados del DataFrame
datos = datos[~datos['Player'].isin(jugadores_lesionados)]



In [69]:
datos.loc[datos['Player'] == 'javonte green', 'PosFantasy'] = 'Back'
datos.loc[datos['Player'] == 'terry rozier', 'PosFantasy'] = 'Back' 

In [70]:

# Crear el problema de optimización
prob = pulp.LpProblem("Optimización_Fantasy", pulp.LpMaximize)

# Crear variables de decisión
jugadores = datos['Player'].tolist()
vars_decision = pulp.LpVariable.dicts("Jugador", jugadores, lowBound=0, upBound=1, cat='Binary')

# Función objetivo
prob += pulp.lpSum([vars_decision[j] * datos.loc[datos['Player'] == j, 'Puntos Fantasy'].values[0] for j in jugadores])

# Restricciones
# Presupuesto total menor o igual a 100
prob += pulp.lpSum([vars_decision[j] * datos.loc[datos['Player'] == j, 'Precio'].values[0] for j in jugadores]) <= 100

# Exactamente 5 jugadores front y 5 back (guard)
prob += pulp.lpSum([vars_decision[j] for j in jugadores if datos.loc[datos['Player'] == j, 'PosFantasy'].values[0] == 'Front']) == 5
prob += pulp.lpSum([vars_decision[j] for j in jugadores if datos.loc[datos['Player'] == j, 'PosFantasy'].values[0] == 'Back']) == 5

# Exactamente 10 jugadores en total
prob += pulp.lpSum([vars_decision[j] for j in jugadores]) == 10

# Nueva restricción: cada jugador se puede seleccionar como máximo una vez
for j in jugadores:
    prob += vars_decision[j] <= 1

# Resolver el problema
status = prob.solve()

# Verificar si se encontró una solución
if pulp.LpStatus[status] != "Optimal":
    print("No se pudo encontrar una solución óptima.")
else:
    # Obtener resultados
    equipo_seleccionado = [j for j in jugadores if vars_decision[j].value() > 0.5]
    puntos_totales = sum(datos.loc[datos['Player'].isin(equipo_seleccionado), 'Puntos Fantasy'])
    costo_total = sum(datos.loc[datos['Player'].isin(equipo_seleccionado), 'Precio'])

    print("Equipo seleccionado:")
    print("{:<25} {:<10} {:<10} {:<15}".format("Jugador", "PosFantasyición", "Precio", "Puntos Fantasy"))
    print("-" * 60)
    for jugador in equipo_seleccionado:
        PosFantasy = datos.loc[datos['Player'] == jugador, 'PosFantasy'].values[0]
        precio = datos.loc[datos['Player'] == jugador, 'Precio'].values[0]
        puntos = datos.loc[datos['Player'] == jugador, 'Puntos Fantasy'].values[0]
        print("{:<25} {:<10} {:<10.2f} {:<15.2f}".format(jugador, PosFantasy, precio, puntos))

    print("\nResumen:")
    print(f"Puntos Fantasy totales: {puntos_totales:.2f}")
    print(f"Costo total: {costo_total:.2f}")

    # Verificar las restricciones
    front_count = sum(1 for j in equipo_seleccionado if datos.loc[datos['Player'] == j, 'PosFantasy'].values[0] == 'Front')
    back_count = sum(1 for j in equipo_seleccionado if datos.loc[datos['Player'] == j, 'PosFantasy'].values[0] == 'Back')

    print(f"\nNúmero de jugadores Front: {front_count}")
    print(f"Número de jugadores Back: {back_count}")
    print(f"Número total de jugadores: {len(equipo_seleccionado)}")
    print(f"¿Se cumple la restricción de presupuesto?: {'Sí' if costo_total <= 100 else 'No'}")

# Verificar si hay inconsistencias en los datos
inconsistencias = datos[datos['PosFantasy'].isin(['Front', 'Back']) == False]
if not inconsistencias.empty:
    print("\nAdvertencia: Se encontraron jugadores con PosFantasyiciones no válidas:")
    print(inconsistencias[['Player', 'PosFantasy']])

Equipo seleccionado:
Jugador                   PosFantasyición Precio     Puntos Fantasy 
------------------------------------------------------------
devin booker              Back       15.00      42.82          
anthony davis             Front      17.00      49.97          
terry rozier              Back       13.00      38.58          
domantas sabonis          Front      17.00      47.88          
dalano banton             Back       8.50       28.66          
simone fontecchio         Front      7.50       24.72          
kenneth lofton jr.        Front      5.00       28.05          
javonte green             Back       6.00       25.20          
lamar stevens             Front      5.50       22.09          
vasilije micic            Back       5.50       21.89          

Resumen:
Puntos Fantasy totales: 329.84
Costo total: 100.00

Número de jugadores Front: 5
Número de jugadores Back: 5
Número total de jugadores: 10
¿Se cumple la restricción de presupuesto?: Sí
